In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import spearmanr, pearsonr
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

cwd = os.getcwd()
# print(cwd)

# # for windownoobs
game_price_prediction_path = os.path.abspath(os.path.join(cwd, '..', '..'))

# # for mac
#game_price_prediction_path = os.path.abspath(os.path.join(cwd))

sys.path.insert(0, os.path.abspath(game_price_prediction_path))

from python_scripts.utilities.api_calls import get_cookie_from_blob, fetch_item_to_df, fetch_items
from python_scripts.sentiment_analysis.config import ITEM, ALL_POLARITY_FILENAME

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""" 
- Takes data from mention_data 
- and a range of others; polarity_data, or fetches price history
- Plots it sexily
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""" 

### Prerequisites


In [ ]:
# Make sure the correct item is imported from config.py

print(f"ITEM = {ITEM}")

In [4]:
# filter_file_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'filter_file.py')
# !python "{filter_file_path}"

# mention_counter_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'mention_counter.py')
# !python "{mention_counter_path}"

# mention_data_combiner_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'mention_data_combiner.py')
# !python "{mention_data_combiner_path}"

# vader_polarity_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'vader_polarity.py')
# !python "{vader_polarity_path}"


### Fetch price and volume history

In [ ]:
### FETCHING ITEM

# Fetch price and volume data FOR ITEM CHOSEN
dailyCookie = get_cookie_from_blob()
items = fetch_items()
df = fetch_item_to_df(ITEM, dailyCookie)
print(f"Data for {ITEM} fetched")


### Plotting compound polarity and volume

In [ ]:
# Load polarity data instead of mentions
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

# Ensure df has a 'date' column
if 'date' not in df.columns:
    df = df.reset_index()

# Merge polarity and volume data
merged_df = pd.merge(polarity_df, df[['date', 'volume']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['compound'], name="Sentiment", line=dict(color='blue')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['volume'], name="Volume", line=dict(color='red')),
    secondary_y=True,
)

# Update layout for better readability
fig.update_layout(
    title_text=f"Sentiment and Volume Over Time for {ITEM}",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Update y-axes
fig.update_yaxes(title_text="Sentiment Score (Compound)", secondary_y=False, gridcolor='lightgrey')
fig.update_yaxes(title_text="Volume Traded", secondary_y=True, gridcolor='lightgrey')

# Update x-axis
fig.update_xaxes(title_text="Date", gridcolor='lightgrey')

# Show the figure
fig.show()

### Plotting compound polarity and price

In [ ]:
# Load polarity data instead of mentions
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

# Ensure df has a 'date' column
if 'date' not in df.columns:
    df = df.reset_index()

# Merge polarity and volume data
merged_df = pd.merge(polarity_df, df[['date', 'price_usd']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['compound'], name="Sentiment", line=dict(color='blue')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['price_usd'], name="Price", line=dict(color='red')),
    secondary_y=True,
)

# Update layout for better readability
fig.update_layout(
    title_text=f"Sentiment and Volume Over Time for {ITEM}",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Update y-axes
fig.update_yaxes(title_text="Sentiment Score (Compound)", secondary_y=False, gridcolor='lightgrey')
fig.update_yaxes(title_text="Volume Traded", secondary_y=True, gridcolor='lightgrey')

# Update x-axis
fig.update_xaxes(title_text="Date", gridcolor='lightgrey')

# Show the figure
fig.show()

### Plotting smoothened compound polarity and smoothened volume

In [ ]:
# Load polarity data instead of mentions
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

# Ensure df has a 'date' column
if 'date' not in df.columns:
    df = df.reset_index()

# Merge polarity and volume data
merged_df = pd.merge(polarity_df, df[['date', 'volume']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Calculate smoothened versions (7-day rolling average)
window_size = 20
merged_df['smooth_compound'] = merged_df['compound'].rolling(window=window_size).mean()
merged_df['smooth_volume'] = merged_df['volume'].rolling(window=window_size).mean()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add smoothened traces
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['smooth_compound'], 
               name="Smoothed Sentiment", line=dict(color='blue', width=2.5)),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['smooth_volume'], 
               name="Smoothed Volume", line=dict(color='red', width=2.5)),
    secondary_y=True,
)

# Update layout for better readability
fig.update_layout(
    title_text=f"Smoothed Sentiment and Volume Over Time for {ITEM} ({window_size}-day moving average)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Update y-axes
fig.update_yaxes(title_text="Sentiment Score (Compound)", secondary_y=False, gridcolor='lightgrey')
fig.update_yaxes(title_text="Volume Traded", secondary_y=True, gridcolor='lightgrey')

# Update x-axis
fig.update_xaxes(title_text="Date", gridcolor='lightgrey')

# Show the figure
fig.show()

### Plotting smoothened compound polarity and smoothened price

In [ ]:
# Load polarity data instead of mentions
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

# Ensure df has a 'date' column
if 'date' not in df.columns:
    df = df.reset_index()

# Merge polarity and price data
merged_df = pd.merge(polarity_df, df[['date', 'price_usd']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Calculate smoothened versions (20-day rolling average)
window_size = 30
merged_df['smooth_compound'] = merged_df['compound'].rolling(window=window_size).mean()
merged_df['smooth_price'] = merged_df['price_usd'].rolling(window=window_size).mean()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add smoothened traces
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['smooth_compound'], 
               name="Smoothed Sentiment", line=dict(color='blue', width=2.5)),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['smooth_price'], 
               name="Smoothed Price", line=dict(color='red', width=2.5)),
    secondary_y=True,
)

# Update layout for better readability
fig.update_layout(
    title_text=f"Smoothed Sentiment and Price Over Time for {ITEM} ({window_size}-day moving average)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Update y-axes
fig.update_yaxes(title_text="Sentiment Score (Compound)", secondary_y=False, gridcolor='lightgrey')
fig.update_yaxes(title_text="Price (USD)", secondary_y=True, gridcolor='lightgrey')

# Update x-axis
fig.update_xaxes(title_text="Date", gridcolor='lightgrey')

# Show the figure
fig.show()

### Plotting compound polarity against volume

In [18]:
# Load and prepare data as before
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

if 'date' not in df.columns:
    df = df.reset_index()

merged_df = pd.merge(polarity_df, df[['date', 'volume']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=merged_df['volume'],
        y=merged_df['compound'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
            opacity=0.6
        ),
        name="Sentiment vs Volume"
    )
)

# Update layout
fig.update_layout(
    title_text=f"Sentiment vs Volume Correlation for {ITEM}",
    xaxis_title="Volume Traded",
    yaxis_title="Sentiment Score (Compound)",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Add gridlines
fig.update_xaxes(gridcolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey')

# Show the figure
fig.show()

# Optional: Calculate and display correlation coefficient
correlation = merged_df['compound'].corr(merged_df['volume'])
print(f"Correlation coefficient between sentiment and volume: {correlation:.3f}")

C:\Users\Nukul\AppData\Local\Temp\ipykernel_21848\1919942114.py:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Correlation coefficient between sentiment and volume: -0.156


### Plotting compound polarity against price

In [20]:
# Load and prepare data as before
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

if 'date' not in df.columns:
    df = df.reset_index()

merged_df = pd.merge(polarity_df, df[['date', 'price_usd']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=merged_df['price_usd'],
        y=merged_df['compound'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
            opacity=0.6
        ),
        name="Sentiment vs Price"
    )
)

# Update layout
fig.update_layout(
    title_text=f"Sentiment vs Price Correlation for {ITEM}",
    xaxis_title="Price (USD)",
    yaxis_title="Sentiment Score (Compound)",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Add gridlines
fig.update_xaxes(gridcolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey')

# Show the figure
fig.show()

# Optional: Calculate and display correlation coefficient
correlation = merged_df['compound'].corr(merged_df['price_usd'])
print(f"Correlation coefficient between sentiment and price: {correlation:.3f}")

C:\Users\Nukul\AppData\Local\Temp\ipykernel_21848\2442556590.py:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Correlation coefficient between sentiment and price: 0.221


### Plotting smoothened compound polarity against smoothened volume

In [29]:
# Load and prepare data as before
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

if 'date' not in df.columns:
    df = df.reset_index()

merged_df = pd.merge(polarity_df, df[['date', 'volume']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Calculate smoothened versions
window_size = 30  # Same window size as previous smoothing
merged_df['smooth_compound'] = merged_df['compound'].rolling(window=window_size).mean()
merged_df['smooth_volume'] = merged_df['volume'].rolling(window=window_size).mean()

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=merged_df['smooth_volume'],
        y=merged_df['smooth_compound'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
            opacity=0.6
        ),
        name="Smoothed Sentiment vs Volume"
    )
)

# Update layout
fig.update_layout(
    title_text=f"Smoothed Sentiment vs Volume Correlation for {ITEM} ({window_size}-day moving average)",
    xaxis_title="Smoothed Volume Traded",
    yaxis_title="Smoothed Sentiment Score (Compound)",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Add gridlines
fig.update_xaxes(gridcolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey')

# Show the figure
fig.show()

# Calculate and display correlation coefficient for smoothed values
correlation = merged_df['smooth_compound'].corr(merged_df['smooth_volume'])
print(f"Correlation coefficient between smoothed sentiment and volume: {correlation:.3f}")

C:\Users\Nukul\AppData\Local\Temp\ipykernel_21848\1606172362.py:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Correlation coefficient between smoothed sentiment and volume: -0.404


### Plotting smoothened compound polarity against smoothened price

In [31]:
# Load and prepare data as before
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

if 'date' not in df.columns:
    df = df.reset_index()

merged_df = pd.merge(polarity_df, df[['date', 'price_usd']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Calculate smoothened versions
window_size = 30  # Same window size as previous smoothing
merged_df['smooth_compound'] = merged_df['compound'].rolling(window=window_size).mean()
merged_df['smooth_price'] = merged_df['price_usd'].rolling(window=window_size).mean()

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=merged_df['smooth_price'],
        y=merged_df['smooth_compound'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
            opacity=0.6
        ),
        name="Smoothed Sentiment vs Price"
    )
)

# Update layout
fig.update_layout(
    title_text=f"Smoothed Sentiment vs Price Correlation for {ITEM} ({window_size}-day moving average)",
    xaxis_title="Smoothed Price (USD)",
    yaxis_title="Smoothed Sentiment Score (Compound)",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Add gridlines
fig.update_xaxes(gridcolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey')

# Show the figure
fig.show()

# Calculate and display correlation coefficient for smoothed values
correlation = merged_df['smooth_compound'].corr(merged_df['smooth_price'])
print(f"Correlation coefficient between smoothed sentiment and price: {correlation:.3f}")

C:\Users\Nukul\AppData\Local\Temp\ipykernel_21848\460366112.py:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Correlation coefficient between smoothed sentiment and price: 0.562


### Calculate stats for compound polarity against volume

In [38]:
# Load and prepare data as before
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

if 'date' not in df.columns:
    df = df.reset_index()

merged_df = pd.merge(polarity_df, df[['date', 'volume']], on='date', how='outer').sort_values('date')
merged_df = merged_df.ffill()

# Calculate smoothened versions with 30-day window
window_size = 30
merged_df['smooth_compound'] = merged_df['compound'].rolling(window=window_size).mean()
merged_df['smooth_volume'] = merged_df['volume'].rolling(window=window_size).mean()

merged_df = merged_df.dropna()

print("Item name: " + ITEM)

# 1. Spearman's Correlation with smoothened data
spearman_corr, spearman_p = spearmanr(merged_df['smooth_compound'], merged_df['smooth_volume'])
print(f"Spearman's correlation (smoothened): {spearman_corr}, p-value: {spearman_p}")

# 2. Pearson's Correlation with smoothened data
pearson_corr, pearson_p = pearsonr(merged_df['smooth_compound'], merged_df['smooth_volume'])
print(f"Pearson's correlation (smoothened): {pearson_corr}, p-value: {pearson_p}")

# 3. Stationarity Test (ADF) for both smoothened series
adf_volume = adfuller(merged_df['smooth_volume'])
adf_compound = adfuller(merged_df['smooth_compound'])

print("\nStationarity Tests (smoothened data):")
print(f"Smoothed Volume - ADF Statistic: {adf_volume[0]}, p-value: {adf_volume[1]}")
print(f"Smoothed Compound - ADF Statistic: {adf_compound[0]}, p-value: {adf_compound[1]}")

Item name: M4A1-S | Golden Coil (Factory New)
Spearman's correlation (smoothened): -0.4979780878496019, p-value: 2.94625272907108e-206
Pearson's correlation (smoothened): -0.4036304254100531, p-value: 1.812738552830476e-129

Stationarity Tests (smoothened data):
Smoothed Volume - ADF Statistic: -1.9766935613674743, p-value: 0.2968847156728128
Smoothed Compound - ADF Statistic: -3.382206554365903, p-value: 0.01158061285198898


### Calculate stats for compound polarity against price

In [39]:
# Load and prepare data as before
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

if 'date' not in df.columns:
    df = df.reset_index()

merged_df = pd.merge(polarity_df, df[['date', 'price_usd']], on='date', how='outer').sort_values('date')
merged_df = merged_df.ffill()

# Calculate smoothened versions with 30-day window
window_size = 30
merged_df['smooth_compound'] = merged_df['compound'].rolling(window=window_size).mean()
merged_df['smooth_price'] = merged_df['price_usd'].rolling(window=window_size).mean()

merged_df = merged_df.dropna()

print("Item name: " + ITEM)

# 1. Spearman's Correlation with smoothened data
spearman_corr, spearman_p = spearmanr(merged_df['smooth_compound'], merged_df['smooth_price'])
print(f"Spearman's correlation (smoothened): {spearman_corr}, p-value: {spearman_p}")

# 2. Pearson's Correlation with smoothened data
pearson_corr, pearson_p = pearsonr(merged_df['smooth_compound'], merged_df['smooth_price'])
print(f"Pearson's correlation (smoothened): {pearson_corr}, p-value: {pearson_p}")

# 3. Stationarity Test (ADF) for both smoothened series
adf_price = adfuller(merged_df['smooth_price'])
adf_compound = adfuller(merged_df['smooth_compound'])

print("\nStationarity Tests (smoothened data):")
print(f"Smoothed Price - ADF Statistic: {adf_price[0]}, p-value: {adf_price[1]}")
print(f"Smoothed Compound - ADF Statistic: {adf_compound[0]}, p-value: {adf_compound[1]}")

Item name: M4A1-S | Golden Coil (Factory New)
Spearman's correlation (smoothened): 0.5853132088725663, p-value: 1.4716850555470572e-302
Pearson's correlation (smoothened): 0.5620118025273062, p-value: 5.18412913784264e-274

Stationarity Tests (smoothened data):
Smoothed Price - ADF Statistic: -0.14317602103240382, p-value: 0.9449645860866956
Smoothed Compound - ADF Statistic: -3.382206554365903, p-value: 0.01158061285198898
